**import**

In [35]:
from path_finder import GreedySearch
from path_finder import BFS
from path_finder import DFS
from path_finder import AStarSearch
from graph_generator import read_input_file
from matplotlib import pyplot as plt
import time
from IPython.display import display

%matplotlib notebook

**test function**

In [ ]:
import timeit
def test(algorithm, start, goal, matrix, is_print=True):
    path_finder = algorithm(start, goal)
    
    start_tick = timeit.default_timer()
    path = path_finder.search(matrix)
    end_tick = timeit.default_timer()
    
    fig = plt.figure()
    fig.canvas.draw()
    PATH_VALUE = 9
    
    if path:
        print("Found path")
        for pos in path:
            matrix[pos] = PATH_VALUE
    else:
        print("Not found path")
    
    if is_print:
        print(f'distance: {len(path)}, time: {end_tick - start_tick}')
        matrix[start] = PATH_VALUE + 2
        matrix[goal] = PATH_VALUE + 2
        fig.canvas.draw()
        plt.imshow(matrix, cmap="tab20c")
        plt.show()

**main**

In [ ]:
# from path_finder import BFS
# from graph_generator import read_input_file

def read_input_and_test(algorithm):
    input_path = 'sample_input.txt'
    matrix, start, goal = read_input_file(input_path)
    if algorithm != None:
        test(algorithm, start, goal, matrix)

if __name__ == '__main__':
#     input_path = 'sample_input.txt'
#     matrix, start, goal = read_input_file(input_path)
    print('BFS')
    read_input_and_test(BFS)
    print('DFS')
    read_input_and_test(DFS)
    print('A')
    read_input_and_test(GreedySearch)
    print('A*')
    read_input_and_test(AStarSearch)

# level 3:

## Collecting all points before reaching the goal

In [36]:
def pretty(d, indent=0):
   for key, value in d.items():
      print('\t' * indent + str(key))
      if isinstance(value, dict):
         pretty(value, indent+1)
      else:
         print('\t' * (indent+1) + str(value))

In [37]:
import heapq
import numpy as np
import random
from path_finder import BasePathFinder
from path_finder import BFS
import json

REACHED_POINT_FLAG = 88

class AllPointSearch():
    def __init__(self, start, goal, list_point, use_heapq = True, algo_to_find_shortest_point = BFS, list_move=[(0, 1), (0, -1), (1, 0), (-1, 0)]):
        '''
            start: start point
            goal: goal point
            list_point: list of points that we have to reach
            algo_to_find_shortest_point: algorithm to find shortest path between given pair of points
            list_move: list of actions
        '''
        
        self.list_point = list_point
        self.start = start
        self.goal = goal
        self.list_move = list_move
        self.list_point = list_point
        self.algo_to_find_shortest_point = algo_to_find_shortest_point
        self.use_heapq = use_heapq
        
        ## this att is used for find path between two points:
        ## path_lookup[start][goal] = ['A', 'B', 'C', ...]
        self.path_lookup = {}
        self.visited = np.zeros((len(list_point), len(list_point)))
    


    # --------------------------------------------------------------------------------
    # CREATE PATH_LOOKUP DICTIONARY

    ### helper function 1
    def _find_shortest_path(self, start_point, end_point, matrix):
        '''
            find shortest path between a pair of points

            input: source, target
            output: result pattern of the correspondent algorithm used for this task
        '''
        shortest_path_finder = self.algo_to_find_shortest_point(start_point, end_point)
        return shortest_path_finder.search(matrix)
    
    ### helper function 2
    def _update_path_lookup(self, head, tail, path):
        if head not in self.path_lookup:
            if self.use_heapq:
                self.path_lookup[head] = []
            else:
                self.path_lookup[head] = {}
        
        dist = 1e9
        if path != False:
            dist = len(path)
        
        if self.use_heapq:
            self.path_lookup[head].append((dist, tail, path))
        else:
            self.path_lookup[head][tail] = (dist, path)

    ### main function
    def _init_path_lookup(self, matrix):
        '''
            Initiate lookup dictionary
            *path_lookup is dictionary:
                key: point1
                value: list of tupple (distance_between_point1_point2, point1, path_from_point1_to_point2)
            
        '''

        matrix[goal] += 1 #mark matrix[goal] != 0: prohibit searching some ways that go through goal point
        for i, start_point in enumerate(list_point):
            shortest_path = self._find_shortest_path(self.start, start_point, matrix)
            self._update_path_lookup(self.start, start_point, shortest_path)
            
            ## find shortest path between each pair of points
            for j in range(i+1, len(list_point)):
                end_point = list_point[j]
                if j == i or self.visited[i, j] == 1 or self.visited[j, i] == 1:
                    continue
                
                shortest_path = self._find_shortest_path(start_point, end_point, matrix)
                self._update_path_lookup(start_point, end_point, shortest_path)

                reverse_shortest_path = shortest_path.copy()
                if shortest_path != False:
                    reverse_shortest_path.reverse()    
                self._update_path_lookup(end_point, start_point, reverse_shortest_path)
                                
                self.visited[i, j] = 1
                self.visited[j, i] = 1
        
        matrix[goal] -= 1 
        
    # ----------------------------------------------------------------------------------
    # SEARCH ORDER OF POINTS TO REACH
    def _lookup_path_of_point(self, start_point):
        '''
            get information about how to reach to other points of a given point
            path_lookup data structure is a priority queue
            
            input: point1
            output: tupple of closest point compare to point1
        '''
        return heapq.heappop(self.path_lookup[start_point])

    def search(self, matrix):
        '''
            start searching: at each point, just choose the closest one

            output:
                final_path: final path from start_point to goal_point. dtype: list of points
                reaching_order: ordinal of reached point in list_point. dtype: list of points
                total_distance: total distance of final_path. dtype: int
        '''
        self._init_path_lookup(matrix)
        
        final_path = [] # list of points that leads to goal
        reaching_order = [] # order of points we will reach - dtype: list of points
        total_distance = 0
        
        start_point = self.start
        num_of_list_point = len(list_point)
        
        while num_of_list_point > 0:
            distance, next_point, path = self._lookup_path_of_point(start_point)
            
            if matrix[next_point] != REACHED_POINT_FLAG:
                total_distance += distance
                final_path += path
                reaching_order+= [next_point]
                
                matrix[next_point] = REACHED_POINT_FLAG
                start_point = next_point
                num_of_list_point-=1

        
        ## find shortest path from the final point to goal
        shortest_path_to_goal = self._find_shortest_path(reaching_order[-1], goal, matrix)

        total_distance += len(shortest_path_to_goal)
        reaching_order.append(goal)
        final_path.extend(shortest_path_to_goal)
        
        return final_path, reaching_order, total_distance

In [38]:
from graph_generator import GraphGenerator

def read_multi_point_input(fpath:str, verbose:bool=False, fill:bool=False):
    """ Read input file with TA Format
    Input:
        fpath: Path to input txt file
        verbose: Print to screen to debug
        fill: Should we fill the polygon
    Output:
        graph: (Numpy array) Matrix to represent graph value
        start: (Tuple of int) Start point
        goal: (Tuple of int) Goal point
    """
    f = open(fpath)
    l = f.readline()
    w, h = [int(x) for x in l.strip().split(',')]

    if verbose:
        print("Read from file....")
        print(f"Width={w}, Height={h}")

    generator = GraphGenerator((h, w), fill=fill)
    
    l = f.readline()
    
    #xStart, yStart, xGoal, yGoal = [int(x) for x in l.strip().split(',')]
    list_coor = l.strip().split(',')
    list_x = list_coor[1::2]
    list_y = list_coor[::2]
    list_x = [int(x) for x in list_x]
    list_y = [int(y) for y in list_y]

    xStart, yStart, xGoal, yGoal = list_x[0], list_y[0], list_x[1], list_y[1]
    list_point = [(list_x[i], list_y[i]) for i in range(2, len(list_x))]
    
    
    if verbose:
        print(f"Start: ({xStart}, {yStart})")
        print(f"Goal: ({xGoal}, {yGoal})")

    num_polygon = int(f.readline().strip())

    if verbose:
        print(f"Found {num_polygon} Polygons")
    for pInd in range(num_polygon):
        l = f.readline()
        point_list = [int(x) for x in l.strip().split(',')]
        polygon = list(zip(point_list[::2], point_list[1::2]))

        generator.add_polygon(polygon)

    f.close()

    if verbose:
        print("Done load input")
        generator.plot_graph()

    return generator.generate_graph(), (xStart, yStart), (xGoal, yGoal), list_point


## main - all points search

In [43]:
file_path = "multi_points.txt"
#file_path = "sample_input.txt"
matrix, start, goal, list_point = read_multi_point_input(file_path)
#matrix, start, goal = read_input_file(file_path)

In [44]:
print(list_point)
print(start)
print(goal)

[(5, 1), (2, 8), (9, 8), (19, 3), (2, 20)]
(1, 1)
(19, 20)


In [46]:
all_point_finder = AllPointSearch(start, goal, list_point, use_heapq=False)

In [49]:
all_point_finder._init_path_lookup(matrix)

21

In [47]:
final_path, reaching_order, total_distance = all_point_finder.search(matrix)

TypeError: heap argument must be a list

In [ ]:
fig = plt.figure()
fig.canvas.draw()
PATH_VALUE = 9

matrix[start] = 7
matrix[goal] = 7
for p in list_point:
    matrix[p] = 8
    
if final_path:
    print("Found path")
    for pos in final_path:
        if matrix[pos] != PATH_VALUE:
            matrix[pos] = PATH_VALUE
        plt.imshow(matrix, cmap="tab20c")
        fig.canvas.draw()
        plt.pause(0.5)
        plt.show()
else:
    print("Not found path")
    

In [6]:
alpha = []
for c in range(65, 65+11, 1):
    alpha.append(c)
    
import itertools
import timeit
start = timeit.default_timer()
for per_a in itertools.permutations(alpha):
    pass
end = timeit.default_timer()

print(end-start)

2.472954553000136


In [31]:
res = []
def permutation(raw_list, prefix_list):
    if len(raw_list) == 0:
        res.append(prefix_list)
        return
    for i in range(len(raw_list)):
        post_list = raw_list[:i] + raw_list[i+1:]
        permutation(post_list, prefix_list + [raw_list[i]])


In [32]:
A = ['a', 'b', 'c', 'd', 'e', 'f']

In [33]:
pref_list = []
permutation(A, pref_list)

[['a', 'b', 'c', 'd', 'e', 'f'],
 ['a', 'b', 'c', 'd', 'f', 'e'],
 ['a', 'b', 'c', 'e', 'd', 'f'],
 ['a', 'b', 'c', 'e', 'f', 'd'],
 ['a', 'b', 'c', 'f', 'd', 'e'],
 ['a', 'b', 'c', 'f', 'e', 'd'],
 ['a', 'b', 'd', 'c', 'e', 'f'],
 ['a', 'b', 'd', 'c', 'f', 'e'],
 ['a', 'b', 'd', 'e', 'c', 'f'],
 ['a', 'b', 'd', 'e', 'f', 'c'],
 ['a', 'b', 'd', 'f', 'c', 'e'],
 ['a', 'b', 'd', 'f', 'e', 'c'],
 ['a', 'b', 'e', 'c', 'd', 'f'],
 ['a', 'b', 'e', 'c', 'f', 'd'],
 ['a', 'b', 'e', 'd', 'c', 'f'],
 ['a', 'b', 'e', 'd', 'f', 'c'],
 ['a', 'b', 'e', 'f', 'c', 'd'],
 ['a', 'b', 'e', 'f', 'd', 'c'],
 ['a', 'b', 'f', 'c', 'd', 'e'],
 ['a', 'b', 'f', 'c', 'e', 'd'],
 ['a', 'b', 'f', 'd', 'c', 'e'],
 ['a', 'b', 'f', 'd', 'e', 'c'],
 ['a', 'b', 'f', 'e', 'c', 'd'],
 ['a', 'b', 'f', 'e', 'd', 'c'],
 ['a', 'c', 'b', 'd', 'e', 'f'],
 ['a', 'c', 'b', 'd', 'f', 'e'],
 ['a', 'c', 'b', 'e', 'd', 'f'],
 ['a', 'c', 'b', 'e', 'f', 'd'],
 ['a', 'c', 'b', 'f', 'd', 'e'],
 ['a', 'c', 'b', 'f', 'e', 'd'],
 ['a', 'c'

In [29]:
pref_list.append(A[2])

In [30]:
pref_list

['a', 'c']